# Primeros pasos para realizar la clasificación supervisada obtenida de un Raster Landcover de la Región de Aysen (2018)


Script trabajado por Francisco Albornoz (2021-04-12) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

Para crear un nuevo ambiente de trabajo realizamos este comando desde nuestro terminal:
    
- `conda create -n gee python`
- `conda activate gee`
- `conda install mamba -c conda-forge`
- `mamba install geemap xarray_leaflet -c conda-forge`

Para correr aquellas librerías no encontradas instalamos con el siguiente comando: (correr borrando # o digitalizando ALT+C)

In [1]:
# !pip install xarray_leaflet
# !pip install geemap

Importamos las librerías a utilizar dentro del siguiente script de Jupyter Notebooks:

  -  os
  - geemap
  - xarray_leaflet
  - ee
  - rasterio
  - numpy
  - matplotlib
  

In [2]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Importar un mapa interactivo

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Insertamos las coordenadas de trabajo, y agregamos una imagen Landsat

In [4]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")

Verificación de datos de la imagen landsat

In [5]:
ee.Date(landsat.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-02-20'

In [6]:
landsat.get('CLOUD_COVER').getInfo()

1.24

## Importar rasters locales

Debemos localizar nuestro raster a subir, verificamos nuestra carpeta de trabajo:

In [7]:
%pwd

'/mnt/c/users/usuario/Documents/GitHub/VitoAlbornoz/GEE'

In [8]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 

In [12]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [13]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [14]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [15]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')
Map

Map(bottom=42006.0, center=[-44, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [16]:
# os.listdir()

In [22]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [23]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [24]:
raster.bounds

BoundingBox(left=-73.169644655, bottom=-46.394042378, right=-70.719691266, top=-43.705745621)

In [25]:
raster.crs

CRS.from_epsg(4326)

In [26]:
raster

<open DatasetReader name='/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif' mode='r'>

## Obteniendo los valores para el Training Datasets

In [27]:
points = landcover.sample(**{
    'region': landsat.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map


Map(bottom=21287.0, center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position', 'tra…

In [28]:
print(points.size().getInfo())

979


In [29]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-71.81733470314191, -45.638247197656]}, 'id': '1', 'properties': {'b1': 4}}


## Entrenando el modelo 

In [30]:
# Usar las bandas para la predicción.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'] #agregar NDVI


# Esta propiedad guarda los valores del landcover 2018
label = 'b1'

# Overlay the points on the imagery to get training.
training = landsat.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [31]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '1_0', 'properties': {'B1': 203, 'B2': 310, 'B3': 670, 'B4': 664, 'B5': 3761, 'B6': 2712, 'B7': 1472, 'b1': 4}}


## Clasificando la Imagen 

In [32]:
# Clasificar las imagenes con las mismas bandas usadas para el entrenamiento
result = landsat.select(bands).classify(trained)

# # Viendo los cluster con colores 
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(bottom=21287.0, center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position', 'tra…

In [33]:
result.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [8021, 8031],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 563985, 0, -30, -4821885]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-73.98100708063204, -44.76775844527536],
    [-74.02202469346787, -44.8756033385409],
    [-74.07633879770931, -45.017837247288014],
    [-74.1620664470257, -45.240975627916335],
    [-74.12979270115957, -45.247229462546805],
    [-73.75657231289094, -45.31840257196503],
    [-72.21717928320084, -45.59721423322858],
    [-71.8122584780602, -45.6666364632359],
    [-71.811454875824, -45.666026316684444],
    [-71.7551818233905, -45.50246632934048],
    [-71.5339811576147, -44.84878097061749],
    [-71.31731944321463, -44.1947660573219],
    [-71.24148928715591, -43.96082893395694],
    [-71.30659742060537, -43.949660914549824],
    [-73.2

## Categorizando según las clases 

In [ ]:
landcover.getInfo()

In [ ]:
class_values = [1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16]

In [10]:
class_palette = ['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94']


In [11]:
landcoverf = result.set('classification_class_values', class_values)
landcoverf = landcoverf.set('classification_class_palette', class_palette)

NameError: name 'result' is not defined

In [ ]:
Map.addLayer(landcoverf, {}, 'Land cover Final' )
Map